In [1]:
%cd ~/dv/sns-chops/resolution/ARCS/
import os, numpy as np

/SNS/users/lj7/dv/sns-chops/resolution/ARCS


In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py

In [3]:
import plotly

In [4]:
import plotly.figure_factory as ff

# Get data

In [5]:
import resolution_plot
reload(resolution_plot)

<IPython.core.display.Javascript object>

<module 'resolution_plot' from 'resolution_plot.pyc'>

In [6]:
fc1data = resolution_plot.ExpData('./V_Cali_Int_Res_FC1_2018_v2.dat')

reading data. please wait...
  done


In [7]:
fc2data = resolution_plot.ExpData('./V_Cali_Int_Res_FC2_2018_v2.dat')

reading data. please wait...
  done


In [8]:
fc1_highres_data = resolution_plot.ExpData('./V_Cali_Int_Res_FC1_HighRes_2018_v2.dat')

reading data. please wait...
  done


In [9]:
unique_nominal_Eis = set( list(fc1data.Ei_list) + list(fc2data.Ei_list) + list(fc1_highres_data.Ei_list) )

In [10]:
unique_nominal_Eis = sorted(list(unique_nominal_Eis))

## Automatic intensity settings

In [11]:
datamap = {1: fc1data, 2: fc2data}

fwhms = list()
ints = list()
Eis = list()
labels = list()
# for Ei, Chopper, Speed, T0 in np.loadtxt('HighIntensitySetting-fixed.txt', delimiter=','):
for Ei, Chopper, Speed, T0 in np.loadtxt('./doug/new_Ei_high_flux.csv', delimiter=','):
    data = datamap[Chopper]
    condition1 = np.logical_and(np.isclose(data.vdata.Energy, Ei), np.isclose(data.chopper_freqs, Speed))
    # print fc1data.vdata.Chopper3[condition1]
    fwhms += list(data.FWHM[condition1])
    ints += list(data.intensity[condition1])
    Eis1 = list(data.vdata.Ei[condition1])
    Eis += Eis1
    labels += ['Chopper %d - Speed %s' % (Chopper, Speed)] * len(Eis1)
    continue
highint_Eis = Eis
highint_ints = ints
highint_FWHMs = fwhms
highint_labels = labels

## High resolution settings data

In [12]:
datamap = {1: fc1data, 2: fc2data}

fwhms = list()
ints = list()
Eis = list()
labels = list()
# for Ei, Chopper, Speed, T0 in np.loadtxt('HighResSetting.txt', delimiter=','):
for Ei, Chopper, Speed, T0 in np.loadtxt('./doug/new_Ei_high_res.csv', delimiter=','):
    data = datamap[Chopper]
    condition1 = np.logical_and(np.isclose(data.vdata.Energy, Ei), np.isclose(data.chopper_freqs, Speed))
    # print fc1data.vdata.Chopper3[condition1]
    fwhms += list(data.FWHM[condition1])
    ints += list(data.intensity[condition1])
    Eis1 = list(data.vdata.Ei[condition1])
    Eis += Eis1
    labels += ['Chopper %d - Speed %s' % (Chopper, Speed)] * len(Eis1)
    continue
highres_Eis = Eis
highres_FWHMs =  fwhms
highres_ints = ints
highres_labels = labels

In [20]:
# highres_labels

# Intensity vs FWHM

In [21]:
Eis = unique_nominal_Eis #[:10]
title='ARCS flux (intensity) vs resolution Version 2 \nuse dropdown list from left to choose incident energy'
data =[]
buttons = []
all_hidden = [False] * len(Eis) * 3 # FC 1 and 2 and FC 1 HighRes
extra_info = dict(
    chopper_freqs = ('nu', '%sHz'),
    FWHM_percentages = ('Resolution percentage', '%.1f%%'),
    RunNumber = ('Run no.', '%d'),
    Chopper3 = ('T0 chopper', '%d')
)
max_res_percentage = 15.
plot_opts = dict(extra_info=extra_info, max_res_percentage=max_res_percentage)
for i, n_Ei in enumerate(Eis):
    plot = fc1data.createPlotXY(n_Ei, 'FWHM', 'intensity', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-1.5'
    data.append(plot)
    #
    plot = fc2data.createPlotXY(n_Ei, 'FWHM', 'intensity', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-100-1.5'
    data.append(plot)
    # 
    plot = fc1_highres_data.createPlotXY(n_Ei, 'FWHM', 'intensity', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-0.5'
    data.append(plot)
    #
    visible = list(all_hidden)
    visible[3*i] = True; visible[3*i+1] = True; visible[3*i+2] = True
    #
    button =  dict(
        label = '%smeV' % n_Ei,
        method = 'update',
        args = [{'visible': visible},
                {'title': 'Ei=%smeV %s' % (n_Ei, title),
                }])
    buttons.append(button)
    continue

updatemenus = list([
    dict(active=-1,
         buttons=list(buttons)
        )
])

atEi = Eis.index(100.)
data[atEi*3].visible = data[atEi*3+1].visible = data[atEi*3+2].visible = True

layout = dict(
    title=title, 
    # showlegend=False, 
    updatemenus=updatemenus,
    xaxis=dict(
        title='FWHM (meV)',
        showspikes=True,
    ),
    yaxis=dict(
        title='Flux -- Peak area (arb. unit)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (intensity) vs resolution: Version 2')

# Peak height vs FWHM

In [22]:
Eis = unique_nominal_Eis #[:10]
title='ARCS flux (peak height) vs resolution Version 2\nuse dropdown list from left to choose incident energy'
data =[]
buttons = []
all_hidden = [False] * len(Eis) * 3 # FC 1 and 2 and FC1-HighRes
extra_info = dict(
    chopper_freqs = ('nu', '%sHz'),
    FWHM_percentages = ('Resolution percentage', '%.1f%%')
)
max_res_percentage = 15.
plot_opts = dict(extra_info=extra_info, max_res_percentage=max_res_percentage)
for i, n_Ei in enumerate(Eis):
    plot = fc1data.createPlotXY(n_Ei, 'FWHM', 'Height', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-1.5'
    data.append(plot)
    #
    plot = fc2data.createPlotXY(n_Ei, 'FWHM', 'Height', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-100-1.5'
    data.append(plot)
    #
    plot = fc1_highres_data.createPlotXY(n_Ei, 'FWHM', 'Height', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-0.5'
    data.append(plot)
    # 
    visible = list(all_hidden)
    visible[3*i] = True; visible[3*i+1] = True; visible[3*i+2] = True
    #
    button =  dict(
        label = '%smeV' % n_Ei,
        method = 'update',
        args = [{'visible': visible},
                {'title': 'Ei=%smeV %s' % (n_Ei, title),
                }])
    buttons.append(button)
    continue

updatemenus = list([
    dict(active=-1,
         buttons=list(buttons)
        )
])

atEi = Eis.index(100.)
data[atEi*3].visible = data[atEi*3+1].visible = data[atEi*3+2].visible = True

layout = dict(
    title=title, 
    # showlegend=False, 
    updatemenus=updatemenus,
    xaxis=dict(
        title='FWHM (meV)',
        showspikes=True,
    ),
    yaxis=dict(
        title='Flux -- Peak height (arb. unit)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (peak height) vs resolution: Version 2')

# FWHM vs Freq

In [23]:
Eis = unique_nominal_Eis #[:10]
title='ARCS resolution vs frequency Version 2 \nuse dropdown list from left to choose incident energy'
data =[]
buttons = []
all_hidden = [False] * len(Eis) * 3 # FC 1 and 2 and FC 1 HighRes
extra_info = dict(
    FWHM_percentages = ('Resolution percentage', '%.1f%%'),
    RunNumber = ('Run no.', '%d'),
    Chopper3 = ('T0 chopper', '%d')
)
max_res_percentage = 15.
plot_opts = dict(extra_info=extra_info, max_res_percentage=max_res_percentage)
for i, n_Ei in enumerate(Eis):
    plot = fc1data.createPlotXY(n_Ei, 'chopper_freqs', 'FWHM', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-100-1.5'
    data.append(plot)
    #
    plot = fc2data.createPlotXY(n_Ei, 'chopper_freqs', 'FWHM', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-1.5'
    data.append(plot)
    # 
    plot = fc1_highres_data.createPlotXY(n_Ei, 'chopper_freqs', 'FWHM', **plot_opts)
    plot.visible = False
    plot.name = 'ARCS-700-0.5'
    data.append(plot)
    #
    visible = list(all_hidden)
    visible[3*i] = True; visible[3*i+1] = True; visible[3*i+2] = True
    #
    button =  dict(
        label = '%smeV' % n_Ei,
        method = 'update',
        args = [{'visible': visible},
                {'title': 'Ei=%smeV %s' % (n_Ei, title),
                }])
    buttons.append(button)
    continue

updatemenus = list([
    dict(active=-1,
         buttons=list(buttons)
        )
])

atEi = Eis.index(100.)
data[atEi*3].visible = data[atEi*3+1].visible = data[atEi*3+2].visible = True

layout = dict(
    title=title, 
    # showlegend=False, 
    updatemenus=updatemenus,
    xaxis=dict(
        title='Chopper Frequency (Hz)',
        showspikes=True,
    ),
    yaxis=dict(
        title='FWHM (meV)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS resolution vs frequency: Version 2')

# {} vs Ei

In [14]:
import plotly, plotly.plotly as py, plotly.figure_factory as ff, plotly.graph_objs as go

In [15]:
# fc1data.vdata.Energy

In [17]:
datalist = [
    dict(name='ARCS-700-1.5', data=fc1data),
    dict(name='ARCS-100-1.5', data=fc2data),
    dict(name='ARCS-700-0.5', data=fc1_highres_data),
]

## Intensity vs Ei

In [18]:
data = []

for dataitem in datalist:
    fcdata = dataitem['data']
    name = dataitem['name']
    labels = ['%s - %sHz' % (name, freq) for freq in fcdata.chopper_freqs ]
    plot = go.Scatter(
        x=fcdata.vdata.Ei,
        y=fcdata.intensity,
        mode='markers',
        text=labels,
        hoverinfo = 'x + y + text'
    )

    plot.visible = True
    plot.name = name
    data.append(plot)
    continue

plot = go.Scatter(
    x=highint_Eis,
    y=highint_ints,
    mode='markers',
    marker=dict(size=10, symbol="triangle-up-open"),
    text=highint_labels,
    hoverinfo = 'x + y + text'
)
plot.visible = True
plot.name = "Auto flux settings"
data.append(plot)

plot = go.Scatter(
    x=highres_Eis,
    y=highres_ints,
    mode='markers',
    marker=dict(size=10, symbol="triangle-down-open"),
    text=highres_labels,
    hoverinfo = 'x + y + text'
)
plot.visible = True
plot.name = "Auto resolution settings"
data.append(plot)


title = 'Intensity vs Incident Energy'
layout = dict(
    title=title, 
    # showlegend=False, 
    # updatemenus=updatemenus,
    xaxis=dict(
        type='log',
        title='Ei (meV)',
        showspikes=True,
    ),
    yaxis=dict(
        title='Flux -- Intensity (arb. unit)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (intensity) vs Ei')

## FWHM vs Ei

In [19]:
data = []

for dataitem in datalist:
    fcdata = dataitem['data']
    name = dataitem['name']
    labels = ['%s - %sHz' % (name, freq) for freq in fcdata.chopper_freqs ]
    plot = go.Scatter(
        x=fcdata.vdata.Ei,
        y=fcdata.FWHM,
        mode='markers',
        text=labels,
        hoverinfo = 'x + y + text'
    )

    plot.visible = True
    plot.name = name
    data.append(plot)
    continue

plot = go.Scatter(
    x=highint_Eis,
    y=highint_FWHMs,
    mode='markers',
    marker=dict(size=10, symbol="triangle-up-open"),
    text=highint_labels,
    hoverinfo = 'x + y + text'
)
plot.visible = True
plot.name = "Auto flux settings"
data.append(plot)

plot = go.Scatter(
    x=highres_Eis,
    y=highres_FWHMs,
    mode='markers',
    marker=dict(size=10, symbol="triangle-down-open"),
    text=highres_labels,
    hoverinfo = 'x + y + text'
)
plot.visible = True
plot.name = "Auto resolution settings"
data.append(plot)

title = 'FWHM vs Incident Energy'
layout = dict(
    title=title, 
    # showlegend=False, 
    # updatemenus=updatemenus,
    xaxis=dict(
        type='log',
        title='Ei (meV)',
        showspikes=True,
    ),
    yaxis=dict(
        type='log',
        title='FWHM (meV)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS resolution (FWHM) vs Ei')

## Peak Height vs Ei

In [20]:
data = []

for dataitem in datalist:
    fcdata = dataitem['data']
    name = dataitem['name']
    labels = ['%s - %sHz' % (name, freq) for freq in fcdata.chopper_freqs ]
    plot = go.Scatter(
        x=fcdata.vdata.Ei,
        y=fcdata.vdata.Height,
        mode='markers',
        text=labels,
        hoverinfo = 'x + y + text'
    )

    plot.visible = True
    plot.name = name
    data.append(plot)
    continue

title = 'Flux (peak height) vs Incident Energy'
layout = dict(
    title=title, 
    # showlegend=False, 
    # updatemenus=updatemenus,
    xaxis=dict(
        type='log',
        title='Ei (meV)',
        showspikes=True,
    ),
    yaxis=dict(
        # type='log',
        title='Flux -- Peak height (arb unit)',
        showspikes=True,
    ),
    hovermode="closest",
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARCS flux (peak height) vs Ei')